# Hypothesis testing

Python activities to complement [*Measurements and their Uncertainties*](http://www.oupcanada.com/catalog/9780199566334.html) (*MU*), Chapter 8, "Hypothesis testing—how good are our models?"

* [Preliminaries](#Preliminaries)
* [The &chi;<sup>2</sup> distribution](#The-$\chi^2$-distribution)
    * [Basic properties](#Basic-properties)
    * [Using &chi;<sup>2</sup> to evaluate a fit](#Using-$\chi^2$-to-evaluate-a-fit)
    * [Exercise 1](#Exercise-1)

## Preliminaries
Before proceeding with this notebook you should review the topics from the [Example Fit notebook](A.1-Example-Fit.ipynb) and read *MU* Ch. 8, "Hypothesis testing—how good are our models?", with the following [goals](https://wiki.its.sfu.ca/departments/phys-students/index.php/Reading_goals_for_Hughes_and_Hase#Hypothesis_testing-how_good_are_our_models.3F) in mind.

1. Be able to explain how to use the *&chi;*<sup>2</sup> statistic to test whether a set of measurements are consistent with a theoretical model, and why it is useful in this context; also, be able to apply the *&chi;*<sup>2</sup> test to assess the agreement between an experiment and a theoretical description of it. Specifically,
     1. be able to state the null hypothesis that is being tested with the *&chi;*<sup>2</sup> test: that the model function *f*(*&theta;*;**x**)  with the best-fit parameters $\mathbf{\theta} = \left\{\theta_1,\ldots,\theta_{\mathcal{N} }\right\}$ and independent variables $\mathbf{x} = \{x_1,\ldots,x_N\}$ is consistent with measurements $\mathbf{y} = \{y_1,\ldots,y_N\}$ that have independently known Gaussian uncertainties $\left\{\alpha_1,\ldots,\alpha_N\right\}$ for each measurement;
     2. be able to explain what the term "degrees of freedom" means in the context of a statistical estimate;
     3. be able to calculate the number of degrees of freedom in a fit;
     4. given *N* data points with known uncertainties and a model with $\mathcal{N}$ parameters that are optimized to yield a goodness-of-fit statistic $\chi_\mathrm{min}^2$, be able to assess the quality of the fit, both qualitatively (by comparing $\chi_\mathrm{min}^2$ with the number of degrees of freedom) and quantitatively [by computing $P(\chi_\mathrm{min}^2;\nu)$].
2. Be able to describe a variety of methods, including the *&chi;*<sup>2</sup> test, to assess the quality of a fit (see Sec. 8.5.3).
3. Recognize that the confidence limits for a parameter estimate will be given by the Student's *t*-distribution when the uncertainty is not known independently, and that this is usually the implicit assumption of curve-fitting software; also, be able to explain why the Student *t*-distribution will approach the Gaussian distribution when the number of degrees of freedom is sufficiently large.
4. Recognize that curve-fitting software often scales fit parameter uncertainties automatically, and how this can lead to erroneous assessments of both the fit quality and the parameter uncertainties (see the two **health warnings** in Sec. 8.9); also, recognize the limited context in which scaling uncertainties is appropriate.

The following code cell includes the initialization commands needed for this notebook.

In [ ]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import chi2

%matplotlib inline

## The $\chi^2$ distribution

### Basic properties
The $\chi^2(x;\nu)$ distribution is what you get when you take $\nu$ random numbers from the normal distribution $\mathcal{N}(x;\mu=0,\sigma^2=1)$ and take the sum of their squares.

In [ ]:
# Seed the RNG
random.seed(0)

# Generate N=1000 sums of squares of nu=10 random numbers
N = 1000
nu = 10
x = random.randn(N,nu)
c2 = np.sum(x**2, axis=1)

# Compute the chi-squared PDF
c2val = np.linspace(0,30)
c2pdf = chi2.pdf(c2val,nu)

# Compare the distribution with the chi-squared PDF
edges = np.arange(30)
plt.hist(c2, bins=edges, range=(0.0, 30.0), density=True, ec='k')
plt.plot(c2val, c2pdf)
plt.xlabel("$\chi^2$")
plt.ylabel("Probability density");

### Using $\chi^2$ to evaluate a fit

When discussing least-squares curve fits the term $\chi^2$ can mean one of three things, depending on the context:
1. The function that we minimize, which involves the data $(\mathbf{x}, \mathbf{y})$, the standard error $\mathbf{\alpha}$, and the model function $f(x;\theta)$ for a set of parameters $\theta$ that we aim to estimate,
$$
\chi^2(\mathbf{\theta};\mathbf{x}, \mathbf{y}, \mathbf{\alpha}) = \sum_{i=1}^N\frac{[y_i - f(x_i;\theta)]^2}{\alpha_i^2};
$$

2. The minimum value of $\chi_\text{min}^2 = \mathop{\text{arg min}}_\theta\chi^2(\mathbf{\theta};\mathbf{x}, \mathbf{y}, \mathbf{\alpha})$ with respect to the parameters $\theta$; and

3. The probability distribution function given by *MU* Eq. (8.3), which we express here in more conventional notation,
$$
\chi^2(x;\nu) = \frac{x^{\frac{\nu}{2}-1}\exp(-x/2)}{2^{\nu/2}\Gamma(\nu/2)}.
$$

Using this terminology to describe the usual procedure for fitting experimental data, we minimize the $\chi^2(\mathbf{\theta};\mathbf{x}, \mathbf{y}, \mathbf{\alpha})$ *function* (1) to obtain the $\chi_\text{min}^2$ *statistic* (2), then compare this statistic to the $\chi^2(x;\nu)$ *distribution* (3) to evaluate the quality of the fit.

Got that? Good. Now let's put it to use. Simulate voltage measurements across a 100 &Omega; resistor for a range of currents, assuming the voltmeter has a 1 mV uncertainty and a 2 mV offset error.

In [ ]:
# Seed the RNG
random.seed(0)

# Simulate voltage across a 100 Ohm resistor
R = 100
I_mA = np.arange(0,110,10)
I = I_mA*1e-3

alphaV = 1e-3
Voff = 2e-3
Vm = R*I + Voff + alphaV*random.randn(np.size(I))

# Plot the data
plt.errorbar(I_mA, Vm, fmt='o')
plt.xlabel("Current (mA)")
plt.ylabel("Voltage (V)");

Now define a model function based on Ohm's law and choose an initial guess for the resistance. The voltage uncertainty is too small to be appear on the `errorbar` plot, so we switch to the conventional `plot` command.

In [ ]:
# Define model
def vOhm(current, resistance):
    return current*resistance

# Assign initial guess for resistance and
RInit = 100
Imodel_mA = np.linspace(0,100)
Imodel = 1e-3*Imodel_mA
Vmodel = vOhm(Imodel, RInit)
plt.plot(I_mA, Vm, 'o')
plt.plot(Imodel_mA, Vmodel, '-')
plt.xlabel("Current (mA)")
plt.ylabel("Voltage (V)");

So far, so good. Now run `curve_fit` to improve the fit by adjusting *R* to minimize the $\chi^2$ function (1).

In [ ]:
R_fit, R_fit_cov = curve_fit(vOhm, I_mA*1e-3, Vm, p0=[RInit], 
                             sigma=alphaV*np.ones(np.size(Vm)), absolute_sigma=True)
alpha_R = np.sqrt(R_fit_cov[0,0])
print(f"R = {R_fit[0]:.3f} ± {alpha_R:.3f} ohms")

Now check the $\chi^2_\text{min}$ statistic (2).

In [ ]:
# Compute and display chi-squared
res = Vm - vOhm(I_mA*1e-3, R_fit)
normres = res/alphaV
chisq = np.sum(normres**2)
print(f"chisq = {chisq:.1f}")

This result signals a problem: we have 11 measurements and 1 fit parameter, so there are 10 statistical degrees of freedom in the fit. The $\chi^2(x;\nu=10)$ distribution (3) has a mean value of 10 and a standard deviation of $\sigma = \sqrt{2\nu}\approx 4.5$, so our result is more than $7\sigma$ above the value expected for such a fit. We can use [`chi2.cdf`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html) to compute the precise probability.

In [ ]:
# Compute the probability of getting this fit result
dof = 10
cdf = chi2.cdf(chisq, dof)
print(f"Cumulative probability = {cdf:.6f}")
print(f"Significance: {1-cdf:.6f}")

So there is a chance of 3 in a million that we would get such a result at random. Let's look at the residuals to see what might be wrong.

In [ ]:
plt.stem(I_mA, normres, use_line_collection=True)
plt.xlabel('Current (mA)')
plt.ylabel('Normalized residual')
plt.xlim(-5,105)
plt.show()

The residuals reveal a strong trend that indicates our model lacks the necessary freedom to achieve a good fit. From the pattern, we can guess that the problem is an offset error. Let's check.

In [ ]:
# Define new model
def vOhm_offset(current, resistance, offset):
    return current*resistance + offset

VoffInit = 0
p_fit, p_fit_cov = curve_fit(vOhm_offset, I_mA*1e-3, Vm, p0=[RInit,VoffInit], 
                             sigma=alphaV*np.ones(np.size(Vm)), absolute_sigma=True)
R_fit = p_fit[0]
Voff_fit = p_fit[1]
alpha_R = np.sqrt(p_fit_cov[0,0])
alpha_Voff = np.sqrt(p_fit_cov[1,1])
print(f"R = {R_fit:.3f} ± {alpha_R:.3f} ohms")
print(f"Voff = {Voff_fit:.3f} ± {alpha_Voff:.3f} V")

Note that the resistance uncertainty increased by about a factor of two. This does *not* mean that the fit is not as good as the first! If the fit quality is better, then the larger uncertainty is actually a more *accurate* estimate of the uncertainty in *R*. Let us check the fit quality now, noting that the extra fit parameter brings the number of statistical degrees of freedom down to 9.

In [ ]:
# Compute and display chi-squared
res_offset = Vm - vOhm_offset(I_mA*1e-3, R_fit, Voff_fit)
normres_offset = res_offset/alphaV
chisq_offset = np.sum(normres_offset**2)
print(f"chisq_offset = {chisq_offset:.1f}")

# Compute the probability of getting this fit result
dof_offset = 9
cdf_offset = chi2.cdf(chisq_offset, dof)
print(f"Cumulative probability = {cdf_offset:.6f}")
print(f"Significance: {1-cdf_offset:.6f}")

Much better! To be sure, though, we need to check that the normalized residuals are consistent with a standard normal distribution.

In [ ]:
plt.stem(I_mA, normres_offset, use_line_collection=True)
plt.xlabel('Current (mA)')
plt.ylabel('Normalized residual')
plt.xlim(-5,105)
plt.show()

No clear pattern emerges from these residuals, so we can accept the fit and report the parameters with their uncertainties with confidence. Note that we can still get an acceptable $\chi^2_\text{min}$ for a poor model if we also overestimate $\alpha$, so it is essential to examine the residuals in addition to $\chi^2_\text{min}$. Also, note that the *reduced* $\chi^2$ statistic [*MU* Eq. (8.6)],
$$
\chi^2_\nu = \frac{\chi^2_\text{min}}{\nu},
$$
has an expectation value of 1, but a standard deviation $\sigma = \sqrt{2/\nu}$, which depends on the number of degrees of freedom. This means that $\chi^2_\nu$ statistic can provide a false sense of security if $\nu$ is large, since the expected deviation from 1 will be small in that case. The reduced $\chi^2$ is useful for quickly checking if a fit for obvious problems, but for quantitative evaluation you should always use the $\chi^2$ cumulative distribution function to compute the significance of your $\chi^2_\text{min}$ statistic for the given number of degrees of freedom.

### Exercise 1
Import the data in `data/pendulum.csv`, which contains simulated measurements of a pendulum period $T$ as a function of its length $L$. Fit the data to the model
$$
T = 2\pi\sqrt{L/g},
$$
where $g$ is the acceleration of gravity. Report the result of the fit, its $1\sigma$ uncertainty, the $\chi^2_\text{min}$ statistic, and the significance of this statistic. Plot the normalized residuals and evaluate their consistency with the assumption of Gaussian random errors with the given $\alpha_T$.

In [ ]:
# Code cell for Exercise 1
# Use this cell for your response, adding cells if necessary.

##### About this notebook
Notebook by J. S. Dodge, 2019. Available from [SFU GitLab](https://gitlab.rcg.sfu.ca/jsdodge/data-analysis-python). The notebook text is licensed under the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License. See more at [Creative Commons](http://creativecommons.org/licenses/by-nc-nd/4.0/). The notebook code is open source under the [MIT License](https://opensource.org/licenses/MIT).